In [77]:
import json
with open ('Raw data', 'r') as raw:
    data = json.load(raw)

In [78]:
print("The participants' profiles are as follows:")
for i in data['participants']:
    print(i)

The participants' profiles are as follows:
{'studied_french_literature': True, 'id': 3, 'education': 'Licence', 'age': '60 ans ou plus', 'hugo_style_familiarity': 'Très familier'}
{'studied_french_literature': False, 'id': 4, 'education': 'Diplôme d’études supérieures (master, doctorat...)', 'age': '50-59', 'hugo_style_familiarity': 'Neutre'}
{'studied_french_literature': True, 'id': 6, 'education': 'Diplôme d’études supérieures (master, doctorat...)', 'age': '30-39', 'hugo_style_familiarity': 'Un peu familier'}
{'studied_french_literature': True, 'id': 8, 'education': 'Licence', 'age': '18-20', 'hugo_style_familiarity': 'Très peu familier'}
{'studied_french_literature': True, 'id': 9, 'education': 'Licence', 'age': '18-20', 'hugo_style_familiarity': 'Neutre'}
{'studied_french_literature': False, 'id': 10, 'education': 'Licence', 'age': '18-20', 'hugo_style_familiarity': 'Peu familier'}
{'studied_french_literature': True, 'id': 11, 'education': 'Diplôme d’études supérieures (master, do

In [79]:
hugo_para = []
print('Below is the list of correct answers:')
for i in data['paragraphs']:
    if i['category'] == 'hugo_paragraphs':
        print(i)
        hugo_para.append(i['id'])           # filtering out the correct answers

print()
print('The ID of the correct answers are:')
print(hugo_para)
print(f'{len(hugo_para)} paragraphs in total')

Below is the list of correct answers:
{'id': 1, 'category': 'hugo_paragraphs', 'text': "Là tout est beau. Les escarpements sombres des deux rives se mirent dans les larges squammes de l'eau. La roideur des pentes fait que la vigne est cultivée sur le [MASQUE] de la même manière que l'olivier sur les côtes de [MASQUE]. Partout où tombe le rayon du midi, si le rocher fait une petite saillie, le paysan y porte à bras des sacs et des paniers de terre, et, dans cette terre, en [MASQUE] il plante un olivier et sur le [MASQUE] il plante un cep. Puis il contre-butte son terrassement avec un mur de pierres sèches qui retient la terre et laisse fuir les eaux. Ici, par surcroît de précaution, pour que les pluies n'entraînent pas la terre, le vigneron la couvre, comme un toit, avec les ardoises brisées de la montagne. De cette façon, au flanc des roches les plus abruptes, la vigne du [MASQUE], comme l'olivier de la [MASQUE], croît sur des espèces de consoles posées au-dessus de la tête du passant 

In [80]:
pid = {}
for i in data['answers']:
    if i['participant_id'] not in pid:
        pid[i['participant_id']] = 1
    else:
        pid[i['participant_id']] += 1

sum_ok = []
print('pid, answer count')
for item in pid.items():
    if item[1] == 14:
        sum_ok.append(item[0])
    print (item)

print()
print('These are the participants that answered 14 questions (the number expected):')
print (sum_ok)

pid, answer count
(3, 16)
(4, 14)
(6, 14)
(8, 14)
(9, 14)
(10, 14)
(11, 15)
(15, 4)
(17, 1)
(18, 14)
(21, 14)
(19, 14)
(20, 14)
(22, 14)
(24, 14)
(25, 14)
(26, 1)
(27, 14)
(28, 2)
(29, 2)
(30, 14)
(31, 3)
(32, 14)
(33, 14)
(34, 14)
(37, 3)
(38, 14)

These are the participants that answered 14 questions (the number expected):
[4, 6, 8, 9, 10, 18, 21, 19, 20, 22, 24, 25, 27, 30, 32, 33, 34, 38]


In [81]:
reg_quiz = []
nodup_q_set = set()
real_q_set = []

(A,B,C,D) = (0,0,0,0)
for p in sum_ok:
    for i in data['answers']:
        if p == i['participant_id']:
            nodup_q_set.add(i['question_id'])
            real_q_set.append(i['question_id'])
            for q in data['questions']:
                if i['question_id'] == q['id']:
                    if q['category'] == 'Hugo VS Other':
                        A += 1
                    elif q['category'] == 'Hugo VS Neutralized':
                        B += 1
                    elif q['category'] == 'Hugo VS Other2Hugo':
                        C += 1
                    elif q['category'] == 'Hugo VS Restored':
                        D += 1
    if (A,B,C,D) == (4,2,4,4) and len(nodup_q_set) == len(real_q_set):
        reg_quiz.append(p)
    (A,B,C,D) = (0,0,0,0)
    nodup_q_set = set()
    real_q_set = []
print(f'These are the only regular quiz sets following the distribution rule: {reg_quiz}')
print(f'Total number of regular quiz sets: {len(reg_quiz)}')

These are the only regular quiz sets following the distribution rule: [4, 6, 18, 21, 25, 30]
Total number of regular quiz sets: 6


In [110]:
(BA, AVR, AA) = (0,0,0)

for i in data['participants']:
        if i['id'] in reg_quiz:
            if i['hugo_style_familiarity'] == 'Peu familier' or i['hugo_style_familiarity'] == 'Très peu familier':
                    BA += 1
            elif (i['hugo_style_familiarity'] == 'Neutre'):
                    AVR += 1
            elif i['hugo_style_familiarity'] == 'Très familier' or i['hugo_style_familiarity'] == 'Un peu familier':
                    AA += 1

pBA = BA/len(reg_quiz)
pAVR = AVR/len(reg_quiz)
pAA = AA/len(reg_quiz)

print(f'Distribution of participants with below average, average and above average Hugo Style Familiarity: {pBA, pAVR, pAA}')

Distribution of participants with below average, average and above average Hugo Style Familiarity: (0.16666666666666666, 0.5, 0.3333333333333333)


In [82]:
print('Here are the valid answers:')
print('qID, [(pID, answer)]')
valid_answers = {}
val_count = 0
for i in data['answers']:
    if i['participant_id'] in reg_quiz:
        val_count += 1
        if i['question_id'] not in valid_answers:
            valid_answers[(i['question_id'])] = [(i['participant_id'], i['choice'])]
        else:
            valid_answers[(i['question_id'])].append((i['participant_id'], i['choice']))
for item in valid_answers.items():
    print (item)
print()
if 14*len(reg_quiz) == val_count:
    print(f'There are indeed, 14*{len(reg_quiz)} = {val_count} valid answers.')
else:
    print('Something went wrong...')

Here are the valid answers:
qID, [(pID, answer)]
(16, [(4, 'right')])
(17, [(4, 'right')])
(18, [(4, 'right')])
(19, [(4, 'left'), (18, 'left')])
(20, [(4, 'right')])
(15, [(4, 'left'), (30, 'right')])
(21, [(4, 'left')])
(22, [(4, 'right')])
(23, [(4, 'right')])
(24, [(4, 'right')])
(13, [(4, 'right')])
(25, [(4, 'right'), (25, 'left')])
(26, [(4, 'left')])
(27, [(4, 'right')])
(28, [(6, 'left')])
(29, [(6, 'left')])
(30, [(6, 'left'), (18, 'left'), (30, 'left')])
(31, [(6, 'left')])
(32, [(6, 'right')])
(33, [(6, 'left')])
(34, [(6, 'left')])
(35, [(6, 'left')])
(36, [(6, 'left')])
(37, [(6, 'right')])
(38, [(6, 'right')])
(39, [(6, 'right')])
(40, [(6, 'left')])
(41, [(6, 'left')])
(96, [(18, 'right')])
(97, [(18, 'right')])
(98, [(18, 'left'), (21, 'right')])
(131, [(21, 'right')])
(99, [(18, 'left')])
(100, [(18, 'right')])
(101, [(18, 'left')])
(102, [(18, 'left')])
(103, [(18, 'right')])
(104, [(18, 'left')])
(105, [(18, 'left')])
(106, [(18, 'right')])
(107, [(18, 'right')])
(1

In [83]:
valid_choices = {}
for i in data['questions']:
    if i['id'] in valid_answers.keys():
        for elem in valid_answers[i['id']]:
            if elem[1] == 'right':
                valid_choices[(elem[0], i['id'])] = i["right_paragraph_id"]
            if elem[1] == 'left':
                valid_choices[(elem[0], i['id'])] = i["left_paragraph_id"]

print()
print('Here are the valid choices:')
print('(pID, qID), paraID')
for item in valid_choices.items():
    print (item)


Here are the valid choices:
(pID, qID), paraID
((21, 5), 15)
((4, 13), 44)
((4, 15), 31)
((30, 15), 30)
((4, 16), 9)
((4, 17), 25)
((4, 18), 5)
((4, 19), 6)
((18, 19), 6)
((4, 20), 42)
((4, 21), 27)
((4, 22), 33)
((4, 23), 48)
((4, 24), 3)
((4, 25), 15)
((25, 25), 14)
((4, 26), 50)
((4, 27), 4)
((6, 28), 52)
((6, 29), 53)
((6, 30), 9)
((18, 30), 9)
((30, 30), 9)
((6, 31), 51)
((6, 32), 12)
((6, 33), 16)
((6, 34), 17)
((6, 35), 27)
((6, 36), 56)
((6, 37), 11)
((6, 38), 33)
((6, 39), 59)
((6, 40), 60)
((6, 41), 4)
((30, 44), 70)
((25, 79), 5)
((21, 80), 95)
((18, 96), 90)
((18, 97), 99)
((18, 98), 19)
((21, 98), 20)
((18, 99), 28)
((18, 100), 76)
((18, 101), 133)
((18, 102), 134)
((18, 103), 135)
((18, 104), 46)
((18, 105), 16)
((18, 106), 34)
((18, 107), 67)
((25, 116), 114)
((21, 116), 113)
((21, 131), 84)
((21, 132), 126)
((21, 133), 34)
((21, 134), 8)
((21, 135), 117)
((21, 136), 119)
((21, 137), 1)
((21, 138), 34)
((21, 139), 70)
((21, 140), 41)
((30, 157), 126)
((25, 163), 1)
((25

In [100]:
total_correct = 0
(A,B,C,D) = (0,0,0,0)
(A_BA, A_A, A_AA) = (0,0,0)
(B_BA, B_A, B_AA) = (0,0,0)
(C_BA, C_A, C_AA) = (0,0,0)
(D_BA, D_A, D_AA) = (0,0,0)


for k,v in valid_choices.items():
    for para in data['paragraphs']:
        if para['id'] == v and para['category'] == 'hugo_paragraphs':
            total_correct += 1
            for q in data['questions']:
                if q['id'] == k[1]:
                    if q['category'] == 'Hugo VS Other':
                        A += 1
                        for i in data['participants']:
                            if i['id'] == k[0]:
                                if i['hugo_style_familiarity'] == 'Peu familier' or i['hugo_style_familiarity'] == 'Très peu familier':
                                    A_BA += 1
                                elif (i['hugo_style_familiarity'] == 'Neutre'):
                                    A_A += 1
                                elif i['hugo_style_familiarity'] == 'Très familier' or i['hugo_style_familiarity'] == 'Un peu familier':
                                    A_AA += 1
                    elif q['category'] == 'Hugo VS Neutralized':
                        B += 1
                        for i in data['participants']:
                            if i['id'] == k[0]:
                                if i['hugo_style_familiarity'] == 'Peu familier' or i['hugo_style_familiarity'] == 'Très peu familier':
                                    B_BA += 1
                                elif (i['hugo_style_familiarity'] == 'Neutre'):
                                    B_A += 1
                                elif i['hugo_style_familiarity'] == 'Très familier' or i['hugo_style_familiarity'] == 'Un peu familier':
                                    B_AA += 1
                    elif q['category'] == 'Hugo VS Other2Hugo':
                        C += 1
                        for i in data['participants']:
                            if i['id'] == k[0]:
                                if i['hugo_style_familiarity'] == 'Peu familier' or i['hugo_style_familiarity'] == 'Très peu familier':
                                    C_BA += 1
                                elif (i['hugo_style_familiarity'] == 'Neutre'):
                                    C_A += 1
                                elif i['hugo_style_familiarity'] == 'Très familier' or i['hugo_style_familiarity'] == 'Un peu familier':
                                    C_AA += 1
                    elif q['category'] == 'Hugo VS Restored':
                        D += 1
                        for i in data['participants']:
                            if i['id'] == k[0]:
                                if i['hugo_style_familiarity'] == 'Peu familier' or i['hugo_style_familiarity'] == 'Très peu familier':
                                    D_BA += 1
                                elif (i['hugo_style_familiarity'] == 'Neutre'):
                                    D_A += 1
                                elif i['hugo_style_familiarity'] == 'Très familier' or i['hugo_style_familiarity'] == 'Un peu familier':
                                    D_AA += 1

print(f'For the fully functional dataset, humans recognized {total_correct} hugo paragraphs out of {val_count} questions (accuracy = {total_correct/val_count}).')
print('More specifically for each text category:')
print(f'A: {A} out of {4*len(reg_quiz)} = {A/(4*len(reg_quiz))}')
print(f'B: {B} out of {2*len(reg_quiz)} = {B/(2*len(reg_quiz))}')
print(f'C: {C} out of {4*len(reg_quiz)} = {C/(4*len(reg_quiz))}')
print(f'D: {D} out of {4*len(reg_quiz)} = {D/(4*len(reg_quiz))}') 

For the fully functional dataset, humans recognized 54 hugo paragraphs out of 84 questions (accuracy = 0.6428571428571429).
More specifically for each text category:
A: 20 out of 24 = 0.8333333333333334
B: 6 out of 12 = 0.5
C: 17 out of 24 = 0.7083333333333334
D: 11 out of 24 = 0.4583333333333333


In [114]:
import pandas as pd

print('Actual Correct Answers')
columns = ['Hugo Style Familiary', 'A', 'B', 'C', 'D']
rows = [('Below Average', A_BA, B_BA, C_BA, D_BA), ('Average', A_A, B_A, C_A, D_A), ('Above Average', A_AA, B_AA, C_AA, D_AA)]
df1 = pd.DataFrame(rows, columns=columns)
df1 = df1.loc[0:2].eval('Sum = A + B + C + D')
df1.loc['total'] = df1.sum(numeric_only=True, axis=0)
print(df1)

Actual Correct Answers
      Hugo Style Familiary     A    B     C     D   Sum
0            Below Average   3.0  0.0   4.0   2.0   9.0
1                  Average  11.0  4.0   6.0   4.0  25.0
2            Above Average   6.0  2.0   7.0   5.0  20.0
total                  NaN  20.0  6.0  17.0  11.0  54.0


In [115]:
print('Question Amount')
columns = ['Hugo Style Familiary', 'A', 'B', 'C', 'D']
rows = [('Below Average', 4*len(reg_quiz)*pBA, 2*len(reg_quiz)*pBA, 4*len(reg_quiz)*pBA, 4*len(reg_quiz)*pBA), ('Average', 4*len(reg_quiz)*pAVR, 2*len(reg_quiz)*pAVR, 4*len(reg_quiz)*pAVR, 4*len(reg_quiz)*pAVR), ('Above Average', 4*len(reg_quiz)*pAA, 2*len(reg_quiz)*pAA, 4*len(reg_quiz)*pAA, 4*len(reg_quiz)*pAA)]
df2 = pd.DataFrame(rows, columns=columns)
df2 = df2.loc[0:2].eval('Sum = A + B + C + D')
df2.loc['total'] = df2.sum(numeric_only=True, axis=0)
print(df2)


Question Amount
      Hugo Style Familiary     A     B     C     D   Sum
0            Below Average   4.0   2.0   4.0   4.0  14.0
1                  Average  12.0   6.0  12.0  12.0  42.0
2            Above Average   8.0   4.0   8.0   8.0  28.0
total                  NaN  24.0  12.0  24.0  24.0  84.0


In [118]:
df1.set_index('Hugo Style Familiary', inplace=True)
df2.set_index('Hugo Style Familiary', inplace=True)

print (df1.div(df2).reset_index())

  Hugo Style Familiary         A         B         C         D       Sum
0        Below Average  0.750000  0.000000  1.000000  0.500000  0.642857
1              Average  0.916667  0.666667  0.500000  0.333333  0.595238
2        Above Average  0.750000  0.500000  0.875000  0.625000  0.714286
3                  NaN  0.833333  0.500000  0.708333  0.458333  0.642857
